<a href="https://colab.research.google.com/github/HeyMahdy/ai-agents-playground/blob/main/ToolAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
%pip install langchain-community==0.3.16
%pip install langchain==0.3.23
%pip install langchain-openai==0.3.14
%pip install -q langgraph==0.2.57

In [85]:
!pip install  langsmith

In [29]:
!pip install -U requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [131]:
import getpass
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [4]:
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, MessageGraph, StateGraph

from typing import List, Sequence
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

In [5]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [7]:
from langchain.tools import tool

In [8]:
from pydantic import BaseModel
from typing import Literal

In [9]:
from pydantic import BaseModel
class Supplier(BaseModel):
    supplier_id: str
    name: str
    location: str
    lead_time_days: int

In [10]:
class Product(BaseModel):
    sku: str
    name: str
    category: str
    unit_of_measure: str
    supplier_id: int

In [44]:
from langchain.tools import tool
import requests
from pydantic import BaseModel, ValidationError


In [51]:
@tool
def add_supply_data(
    supplier_id: str,
    name: str,
    location: str,
    lead_time_days: int
):
    """
    Send supplier data in database to save it.

    This function acts as a tool to submit supplier information to the
    FastAPI backend. It validates the input data using the `Supplier` Pydantic
    model, sends a POST request to the endpoint, and handles possible errors
    gracefully.

    Args:
        supplier_id (str): Unique identifier for the supplier.
        name (str): Name of the supplier.
        location (str): Physical location of the supplier.
        lead_time_days (int): Average lead time for the supplier's products in days.

    Returns:
        dict: The JSON response from the FastAPI server if the request is
        successful. The successful return format is:

        {
            "supplier_id": "string",
            "name": "string",
            "location": "string",
            "lead_time_days": 0,
            "id": 0
        }

        In case of errors, returns a dictionary with an 'error' key describing
        the problem. Possible error cases:
            - ValidationError: Input data is invalid.
            - HTTPError: The API returned a bad HTTP status code (e.g., 405).
            - RequestException: Network or request failure (e.g., timeout).


    """
    try:
        # Validate input using Pydantic
        supplier = Supplier(
            supplier_id=supplier_id,
            name=name,
            location=location,
            lead_time_days=lead_time_days
        )
        payload = supplier.model_dump()  # get dict representation

        # Make POST request to FastAPI
        url = "https://factoryai.onrender.com/suppliers/"
        headers = {"Content-Type": "application/json"}
        response = requests.post(url, json=payload, headers=headers, timeout=10)

        # Raise exception for bad HTTP status codes
        response.raise_for_status()

        # Return JSON response
        return response.json()

    except ValidationError as ve:
        return {"error": "Invalid supplier data", "details": ve.errors()}
    except requests.exceptions.HTTPError as he:
        return {"error": "HTTP error occurred", "status_code": response.status_code, "details": str(he)}
    except requests.exceptions.RequestException as re:
        return {"error": "Request failed", "details": str(re)}

In [46]:
@tool
def get_supply_data():
    """
    Retrieve the list of suppliers database.

    This function sends a GET request to the FastAPI backend to fetch all
    suppliers. It handles HTTP errors and network issues gracefully.

    Returns:
        list[dict]: A list of supplier objects with the following format on
        successful request:

        [
            {
                "supplier_id": "string",
                "name": "string",
                "location": "string",
                "lead_time_days": 0,
                "id": 0
            }
        ]

        In case of errors, returns a dictionary with an 'error' key describing
        the problem. Possible error cases:
            - HTTPError: The API returned a bad HTTP status code (e.g., 405).
            - RequestException: Network or request failure (e.g., timeout).
    """
    try:

        # Make POST request to FastAPI
        url = "https://factoryai.onrender.com/suppliers/"
        headers = {"Content-Type": "application/json"}
        response = requests.get(url, headers=headers, timeout=10)

        # Raise exception for bad HTTP status codes
        response.raise_for_status()

        # Return JSON response
        return response.json()

    except ValidationError as ve:
        return {"error": "Invalid supplier data", "details": ve.errors()}
    except requests.exceptions.HTTPError as he:
        return {"error": "HTTP error occurred", "status_code": response.status_code, "details": str(he)}
    except requests.exceptions.RequestException as re:
        return {"error": "Request failed", "details": str(re)}

In [53]:
@tool
def add_product_data(
    sku: str,
    name: str,
    category: str,
    unit_of_measure: str,
    supplier_id: int
):
    """
    Send product data to database to store it.

    This function acts as a tool to submit a new product to the FastAPI
    backend. It validates input data using the `Product` Pydantic model,
    sends a POST request to the endpoint, and handles errors gracefully.

    Args:
        sku (str): Unique SKU identifier for the product.
        name (str): Name of the product.
        category (str): Category or type of the product.
        unit_of_measure (str): Unit in which the product is measured (e.g., kg, pcs).
        supplier_id (int): ID of the supplier associated with this product.

    Returns:
        dict: The JSON response from the FastAPI server if successful. The
        response format typically includes the product fields and its assigned
        database ID. Example:

        {
            "sku": "string",
            "name": "string",
            "category": "string",
            "unit_of_measure": "string",
            "supplier_id": 0,
            "id": 0
        }

        In case of errors, returns a dictionary with an 'error' key describing
        the problem. Possible error cases:
            - ValidationError: Input data is invalid.
            - HTTPError: The API returned a bad HTTP status code (e.g., 405).
            - RequestException: Network or request failure (e.g., timeout).

    """
    try:
        # Validate input using Pydantic
        supplier = Supplier(
            sku=sku,
            name=name,
            category=category,
            unit_of_measure=unit_of_measure,
            supplier_id=supplier_id

        )
        payload = supplier.model_dump()  # get dict representation

        # Make POST request to FastAPI
        url = "https://factoryai.onrender.com/products/"
        headers = {"Content-Type": "application/json"}
        response = requests.post(url, json=payload, headers=headers, timeout=10)

        # Raise exception for bad HTTP status codes
        response.raise_for_status()

        # Return JSON response
        return response.json()

    except ValidationError as ve:
        return {"error": "Invalid supplier data", "details": ve.errors()}
    except requests.exceptions.HTTPError as he:
        return {"error": "HTTP error occurred", "status_code": response.status_code, "details": str(he)}
    except requests.exceptions.RequestException as re:
        return {"error": "Request failed", "details": str(re)}

In [109]:
@tool
def get_product_data():
    """
    Retrieve the list of products from the database.

    This function sends a GET request to the FastAPI `/products/` endpoint
    to fetch all products. It handles HTTP errors and network issues gracefully.

    Returns:
        list[dict]: A list of product objects with the following format on
        successful request:

        [
            {
                "sku": "string",
                "name": "string",
                "category": "string",
                "unit_of_measure": "string",
                "supplier_id": 0,
                "id": 0
            }
        ]

        In case of errors, returns a dictionary with an 'error' key describing
        the problem. Possible error cases:
            - HTTPError: The API returned a bad HTTP status code (e.g., 405).
            - RequestException: Network or request failure (e.g., timeout).

    """
    try:

        # Make POST request to FastAPI
        url = "https://factoryai.onrender.com/products/"
        headers = {"Content-Type": "application/json"}
        response = requests.get(url,params="",headers=headers, timeout=10)

        # Raise exception for bad HTTP status codes
        response.raise_for_status()

        # Return JSON response
        return response.json()

    except ValidationError as ve:
        return {"error": "Invalid supplier data", "details": ve.errors()}
    except requests.exceptions.HTTPError as he:
        return {"error": "HTTP error occurred", "status_code": response.status_code, "details": str(he)}
    except requests.exceptions.RequestException as re:
        return {"error": "Request failed", "details": str(re)}

In [110]:
respon = get_product_data.invoke(input)

In [111]:
print(respon)

[{'sku': 'string', 'name': 'string', 'category': 'string', 'unit_of_measure': 'string', 'supplier_id': 6, 'id': 2}]


In [57]:
tools=[add_supply_data,get_product_data,get_supply_data,add_product_data]
tools_by_name={ tool.name:tool for tool in tools}

In [106]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import MessagesPlaceholder , ChatPromptTemplate

system_message = ChatPromptTemplate.from_messages([
    ("system","""

    You are a helpful AI assistant that thinks step-by-step and uses tools when needed.

When responding to queries:
1. First, think carefully about what information you need.
2. Use available tools if you require current data or specific capabilities.
3. Provide clear, accurate, and helpful responses based on your reasoning and any tool results.

Always explain your thinking process so the user can understand your approach.
Be thorough, logical, and structured in your answers.
Only fetch product information if user wants products data. Do NOT fetch supplier data
also Only fetch supply information if user wants supply data. Do NOT products supplier data



    """),
   MessagesPlaceholder(variable_name='scratch_pad')

])

In [107]:
model_react=system_message|llm.bind_tools(tools)

In [58]:
from typing import (Annotated,Sequence,TypedDict)
from langchain_core.messages import BaseMessage,ToolMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """The state of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [68]:
def call_model(state: AgentState):
    """Invoke the model with the current conversation state."""
    response = model_react.invoke({"scratch_pad": state["messages"]})
    return {"messages": [response]}

In [124]:
import json

In [126]:
from langchain_core.messages import ToolMessage
def tool_node(state: AgentState):
    outputs = []

    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])

        # tool_result is already a list of dicts
        if isinstance(tool_result, list):
            # Convert to JSON string, so LLM treats it as content
            content = {"text": json.dumps(tool_result)}
        else:
            content = {"text": str(tool_result)}

        outputs.append(
            ToolMessage(
                content=content,  # keep as list of dicts
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": outputs}


In [70]:
def should_continue(state: AgentState):
    """Determine whether to continue with tool use or end the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"


In [127]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow4 = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow4.add_node("agent", call_model)
workflow4.add_node("tools", tool_node)

# Add edges between nodes
workflow4.add_edge("tools", "agent")  # After tools, always go back to agent

# Add conditional logic
workflow4.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",  # If tools needed, go to tools node
        "end": END,          # If done, end the conversation
    },
)

# Set entry point
workflow4.set_entry_point("agent")

# Compile the graph
graph = workflow4.compile()

In [132]:
def print_stream(stream):
    """Helper function for formatting the stream nicely."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [HumanMessage(content="Add a supplier with ID 'riheu', name 'nveriuvne', located in 'New York', and a lead time of 5 days?")]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add a supplier with ID 'riheu', name 'nveriuvne', located in 'New York', and a lead time of 5 days?
================================== Ai Message ==================================
Tool Calls:
  add_supply_data (call_yW9OZzGLRvDxAwPwLJUMtB0r)
 Call ID: call_yW9OZzGLRvDxAwPwLJUMtB0r
  Args:
    supplier_id: riheu
    name: nveriuvne
    location: New York
    lead_time_days: 5
================================= Tool Message =================================
Name: add_supply_data

{'text': '{\'error\': \'Request failed\', \'details\': "HTTPSConnectionPool(host=\'factoryai.onrender.com\', port=443): Read timed out. (read timeout=10)"}'}
================================== Ai Message ==================================

It seems that there was a timeout issue when trying to add the supplier data. This can happen due to network issues, server unavailability, or high server load.

Would you like me to try adding t